In [26]:
import os
import sys

from snowflake.snowpark.types import StringType
from snowflake.snowpark.functions import udf


from snowflake.snowpark import Session
import pandas as pd
from openai import OpenAI


openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)



In [27]:
# Unpack env vars
API_BASE = "http://localhost:8000/v1/completions"
SNOW_ROLE = os.environ["SNOW_ROLE"]
SNOW_WAREHOUSE = os.environ["SNOW_WAREHOUSE"]
SNOW_DATABASE = os.environ["SNOW_DATABASE"]
SNOW_SCHEMA = os.environ["SNOW_SCHEMA"]
SNOW_TABLE = "MISTRAL_VLLM_DB.PUBLIC.PRODUCT_INFO" 


# Establish snowpark session
def get_token():
    with open('/snowflake/session/token', 'r') as f:
        return f.read()

connection_params = {
    'host': os.environ['SNOWFLAKE_HOST'],
    'port': os.environ['SNOWFLAKE_PORT'],
    'protocol': 'https',
    'account': os.environ['SNOWFLAKE_ACCOUNT'],
    'authenticator': 'oauth',
    'token': get_token(),
    'role': SNOW_ROLE,
    'warehouse': SNOW_WAREHOUSE,
    'database': SNOW_DATABASE,
    'schema': SNOW_SCHEMA,
     'session_parameters':{
            'PYTHON_CONNECTOR_QUERY_RESULT_FORMAT': 'json'
        }
}


session = Session.builder.configs(connection_params).create()
token = get_token()



In [37]:
@udf(name="build_prompts", is_permanent=True, stage_location="@files", replace=True, session=session)
def build_prompt(name: str, feature_1: str, feature_2: str, target_customer1: str,feature_3: str) -> str:
    PROMPT_TEMPLATE = '''<s>[INST] Please generate a online store product description for a {0} that highlights {1}, {2} and {4}. Personalize the description for {3} [/INST]'''
    return PROMPT_TEMPLATE.format(name, feature_1, feature_2, target_customer1,feature_3)

def get_df(table_name, n = None):
    df = session.table(table_name)
    df_new= df.select(df["id"],df["name"], build_prompt(df["name"], df["feature_1"], df["feature_2"], df["target_customer1"], df["feature_3"]).as_("prompt"))
    return df_new.select("id", "name", "prompt")


In [42]:
df_s = get_df("MISTRAL_VLLM_DB.PUBLIC.PRODUCT_INFO",
            table_sample).collect()
df = pd.DataFrame(df_s)
df.head()

,ID,NAME,PROMPT
0,A1234,EJOffice Standup Desk,<s>[INST] Please generate a online store produ...
1,A3214,EJOffice Chair,<s>[INST] Please generate a online store produ...
2,A3215,EJOffice Regular Desk,<s>[INST] Please generate a online store produ...
3,A3216,EJOffice Trash Wastebasket,<s>[INST] Please generate a online store produ...
4,A3217,EJOffice Paper Shredder,<s>[INST] Please generate a online store produ...


In [43]:
prompt_list=df["PROMPT"].values.tolist()

outputs = [client.completions.create(model="mistralai/Mistral-7B-Instruct-v0.2",
                                      prompt=p, 
                                      max_tokens=200,
                                      n=1,
                                      stop=None,
                                      temperature=0.9) for p in prompt_list]

descriptions = []
# Print the outputs.
for output in outputs:
        generated_text = output.choices[0].text
        descriptions.append(generated_text.strip())
        print(f"Generated text: {generated_text!r}")
df['DESCRIPTION'] = descriptions

Generated text: " Title: EJOffice Stand-Up Desk: Your Ergonomic and Affordable Remote Office Solution\n\nSubtitle: Upgrade Your Home Office with Our Solid Wood, Height-Adjustable Standing Desk - Designed for Comfort and Productivity\n\nDescription:\n\nWelcome, remote office warriors! Are you feeling the strain of long hours sitting at your makeshift desk, hunched over a laptop in a cluttered corner of your home? We've got the solution for you: the EJOffice Stand-Up Desk, the perfect fusion of ergonomics, affordability, and natural beauty.\n\nWhy is the EJOffice Stand-Up Desk the best investment for your remote work setup? Let us count the ways:\n\n1. Ergonomic design: Our desktop is designed to help you maintain optimal posture while working, reducing the risk of back pain, neck pain"
Generated text: " Product Name: EJ Remote Work Comfort Chair - Ergonomic, Adjustable, and Luxurious Leather Seat for Home Offices\n\nWelcome to the future of home officing with our EJ Remote Work Comfort 

In [44]:
DESTINATION_TABLE = "PRODUCT_DESCRIPTIONS"
DESTINATION_COLUMNS = 'ID,NAME,DESCRIPTION'
 
_ = session.write_pandas(df[DESTINATION_COLUMNS.split(',')], 
                           DESTINATION_TABLE, 
                           auto_create_table=True, 
                           overwrite=True,
                           on_error='continue')